In [180]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import csv
import random
from bs4 import BeautifulSoup
import progressbar
import numpy as np
from collections import deque
import json

In [75]:
paths_common = ['GESTION/NOJO',
                'GESTION/TETIER_R4',
                'GESTION/NOM_HTML',
                'DONNEES/IDENT/NOM',
                'DONNEES/IDENT/PRM',
                'DONNEES/IDENT/ADRESSE',
                'DONNEES/IDENT/CP',
                'DONNEES/IDENT/VILLE',
                'DONNEES/IDENT/MEL',
                'DONNEES/IDENT/TEL',
                'DONNEES/OBJET/OBJET_COMPLET'
                ]

#give_me_the_tag = ['DONNEES/INFO',
                   #'DONNEES/TYPE_ACTIVITE_ORG']

paths_take_all = ['DONNEES/CLASSES',
                  'DONNEES/DESCRIPTEURS',
                  'DONNEES/CARACTERISTIQUES']

jump_a_line = ['DONNEES/PROCEDURES/LOTS']

tags = []
for path in paths_common:
    tags.append(path2tag(path))
for path in paths_take_all:
    tags.append(path2tag(path))
tags.append('LOTS')

In [69]:
def path2tag(path):
    pavel = ''
    for i in reversed(path):
        if i != '/':
            pavel += i
        if i == '/':
            break
    return pavel[::-1]

In [181]:
def give_me_your_dict(annonce):
    lots = []
    out = {}
    if annonce.find('DONNEES/PROCEDURES/LOTS') is not None:
        reslit = list(annonce.find('DONNEES/PROCEDURES/LOTS').iter())
    else:
        return out
    
    if reslit is not None:
        for element in reslit:
            if element.tag == 'NUM_LOT' and element.text is not None:
                sub_list = []
                for lot_element in reslit[reslit.index(element):]:
                    sub_list.append(lot_element)
                    if reslit.index(lot_element) + 1 < len(reslit):
                        next_element = reslit[reslit.index(lot_element) + 1]
                
                        if next_element.tag == 'NUM_LOT' and next_element.text is not None:
                            lots.append(sub_list)
                            break
    for arr in lots:
        out[lots.index(arr)] = {element.tag: element.text for element in arr}
    
    out = json.dumps(out, ensure_ascii=False)
        
    return out

In [132]:
def xml_parser(paths_common, paths_take_all, file):
    rowsamarr = []
    rows = []
    
    for path in paths_common:
        element = file.find(path)
        if element is not None:
            if element.text is not None:
                rows.append(element.text)
            else:
                rows.append('.')
        elif element is None:
            rows.append('.')
    """
    for path in give_me_the_tag:
        element = file.find(path)
        pavel = ''
        if element is not None:
            for child in element.getchildren():
                if child is not None:
                    pavel += child.tag
            pairs.append([path2tag(path), pavel])
        elif element is None:
            pairs.append([path2tag(path), ''])
    """     
    
    for path in paths_take_all:
        element = file.find(path)
        if element is not None:
            reslist = list(element.iter()) 
            result = ' '.join([element.text for element in reslist if element.text is not None])
            rows.append(result)
        elif element is None:
            rows.append('.')
    
    lots = give_me_your_dict(file)
    if lots is not None:
        rows.append(lots)
    else: 
        rows.append('.')
    return rows, tags 
"""  
    if len(lots) > 0:
        rows.append(lots[0])
        rowsamarr.append(rows)
        if len(lots) > 1:
            for lot in lots[1:]:
                row = ['.' for i in range(len(tags) - 1)]
                row.append(lot)
                rowsamarr.append(row)
    else:
        rowsamarr.append(rows)
"""        
#  return rows, tags 


"  \n    if len(lots) > 0:\n        rows.append(lots[0])\n        rowsamarr.append(rows)\n        if len(lots) > 1:\n            for lot in lots[1:]:\n                row = ['.' for i in range(len(tags) - 1)]\n                row.append(lot)\n                rowsamarr.append(row)\n    else:\n        rowsamarr.append(rows)\n"

In [178]:
def write_to_csv(big_row_list, file_name, tags, year):
    
    with open('{}.csv'.format(file_name), 'w') as file:
        spamwriter = csv.writer(file, delimiter=';', quotechar='"',
                            quoting=csv.QUOTE_ALL)
        spamwriter.writerow(tags)
        for row in big_row_list:
            spamwriter.writerow(row)
        print('wrote {} rows for year {}'.format(len(big_row_list), year))
        
            

In [144]:
def pbar(maxlen, text=''):
    
    bar = progressbar.ProgressBar(maxval=maxlen, \
    widgets=[progressbar.Bar('=', '[', ']'), text, progressbar.Percentage(), ' (', progressbar.ETA(), ') '])
    
    bar.start()
    
    return bar

In [177]:
def write_years2csv(start, end):
    
    years = [start + i for i in range(end - start + 1)]

    
    for i in range(len(years)):
        paths = glob.glob('{}/mp*/*.xml'.format(years[i]))
        #paths = glob.glob('rand/*.xml')
        annonces = []
    
        bar = pbar(len(paths), ' reading {} data'.format(years[i]))
        
    
        for path in paths:
            bar.update(paths.index(path))
            file = open(path, encoding='Latin-1')
            apple = file.read()

            apple = apple.replace('&apos;', "'")
            apple = apple.replace('&deg;', '°')
            apple = apple.replace('&quot;', '"')
            apple = apple.replace('\t', '')
            apple = apple.replace(',', '')
            apple = apple.replace('\n', '')
            apple = apple.replace('é', 'e')
            apple = apple.replace('ç', 'c')
            apple = apple.replace('è', 'e')
            apple = apple.replace(':', '')
            apple = apple.replace('à', 'a')
            apple = apple.replace('ù', 'u')
            apple = apple.replace('ê', 'e') #il y a forcément un moyen plus simple

            tree = ET.fromstring(apple)
            a = tree.getchildren()
    
            annonces.append(a)
    
        pavel = []
        bar2 = pbar(len(annonces), ' parsing {} data'.format(years[i]))
        for annonce in annonces:
            bar2.update(annonces.index(annonce))
            for j in range(len(annonce)):
                extracted_annonce, tags = xml_parser(paths_common, paths_take_all, annonce[j])
                pavel.append(extracted_annonce)
                
        write_to_csv(pavel, years[i], tags, years[i])


    

In [182]:
write_years2csv(2005, 2014)

[==================================== ] parsing 2005 data 99% (ETA:   0:00:00) 

wrote 181247 rows for year 2005


[==================================== ] parsing 2006 data 99% (ETA:   0:00:00) 

wrote 200116 rows for year 2006


[==================================== ] parsing 2007 data 99% (ETA:   0:00:00) 

...

wrote 178858 rows for year 2012


[==================================== ] parsing 2013 data 99% (ETA:   0:00:00) 

wrote 175458 rows for year 2013


[==================================== ] parsing 2014 data 99% (ETA:   0:00:00) 

wrote 165084 rows for year 2014


In [163]:
a = 'apojfezeijg;;;;'

In [164]:
a.replace(';', '')

'apojfezeijg'